# 📘 **Introduction: Feature-Rich Market Data for ML & Deep Learning Models**

This notebook builds a **high-quality, feature-engineered dataset** for forecasting NIFTY price movements using Machine Learning and Deep Learning architectures.

The workflow transforms raw OHLCV market data into a rich, multidimensional feature matrix by adding **Trend**, **Momentum**, **Volatility**, and **Volume-based** indicators.

---

## 🔹 **1. Data Source**

We fetch historical **NIFTY Index or Stock** price data using Yahoo Finance (2012–2025), including:

- Open  
- High  
- Low  
- Close  
- Volume  

The data is cleaned, normalized, and prepared for technical indicator augmentation.

---

## 🔹 **2. Technical Indicator Engineering**

To enhance predictive performance, the dataset includes indicators from **four major categories**, providing the model with deep insights into market behavior. Two categories: Trend and Momemtum integrated in this case study.

---

## 🟦 **(1) Trend Indicators**

Trend indicators help identify the **direction**, **strength**, and **continuation** of price movements.

Included trend features:

- SMA20, EMA20, WMA20, HMA20  
- VWMA20  
- MACD, MACD Signal, MACD Histogram  
- ADX, +DI, –DI  
- Aroon Up, Aroon Down  
- Parabolic SAR  
- TRIX  
- KST  
- Mass Index  
- Vortex (+), Vortex (–)  
- DPO  
- TEMA  

These indicators help models understand persistent directional dynamics and trend reversals.

---

## 🟩 **(2) Momentum Indicators**

Momentum indicators quantify the **speed**, **acceleration**, and **strength** of price movement.

Included momentum features:

- RSI  
- Stochastic %K, Stochastic %D  
- Stochastic RSI  
- Williams %R  
- Rate of Change (ROC)  
- Momentum  
- Ultimate Oscillator  
- Awesome Oscillator  
- PPO  
- PVO  
- TRIX  
- KST  
- TSI  
- KDJ  

These indicators help ML/DL models capture short-term volatility, exhaustion zones, and breakout strength.

---

## 🟧 **(3) Volatility Indicators**  
*(Framework-ready—can be added anytime)*

Volatility indicators measure risk, compression, and expansion phases:

- ATR & ATR Bands  
- Bollinger Bands (mid, upper, lower, width)  
- Keltner Channels  
- Donchian Channels  
- Ulcer Index  
- Parkinson Volatility  
- Garman–Klass Estimator  
- Coefficient of Variation (CV)

These indicators enable models to detect **volatility regimes**, **stress periods**, and **breakout conditions**.

---

## 🟨 **(4) Volume-Driven Indicators**  
*(Partially included—VWMA20 already added)*

Volume indicators provide insight into the **strength** behind price movements:

- VWMA  
- Volume Rate of Change  
- On-Balance Volume (OBV)  
- Money Flow Index (MFI)  
- Volume Oscillators  

Volume-based analysis helps detect **accumulation**, **distribution**, and **institutional involvement**.

---

## 📊 **3. Combined Feature Dataset**

After merging OHLCV with trend and momentum indicators, the final dataset contains:

- **43 engineered features**
- **3,154 clean and aligned rows**
- Fully NaN-free and model-ready

This provides a robust feature space ideal for forecasting tasks based on sequential or pattern-recognition models.

---

## 🚀 **4. Why This Dataset Enhances Model Accuracy**

Market data is complex, nonlinear, and often noisy. Raw OHLCV alone is insufficient for strong predictive performance.  
By integrating diverse technical indicators:

- **Trend indicators** → model learns continuation or reversal patterns  
- **Momentum indicators** → captures short-term strength and exhaustion  
- **Volatility indicators** → identifies breakout and compression phases  
- **Volume indicators** → confirms moves and detects accumulation/distribution  

This makes the dataset **richer, more expressive, and far more predictive**, ideal for:

- LSTM / GRU / Transformer sequence models  
- Gradient boosting models (XGBoost, LightGBM, CatBoost)  
- Reinforcement learning systems  
- Backtesting & algorithmic trading research  

---

This structured and feature-rich dataset provides the ideal foundation for building accurate, robust financial prediction models.


In [29]:
# ============================================
# 1. Install Libraries
# ============================================
!pip install yfinance ta numpy pandas scikit-learn

import yfinance as yf
import pandas as pd
import numpy as np
from ta import momentum, trend, volatility
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# ============================================
# 2. Load NIFTY Futures OHLCV data
# Ticker for NIFTY Futures on Yahoo → ^NSEI or ^NSEBANK or similar.
# You can replace with your broker’s data as needed.
# ============================================

ticker = "RELIANCE.NS"   # NIFTY Index (not futures, but behaves similarly for modeling)
df = yf.download(ticker, start="2012-01-01", end="2025-01-01")

# Keep only OHLCV
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
df.dropna(inplace=True)

print("Data Loaded:", df.shape)
df.head()

# ============================================
# 3. Add Technical Indicators (RSI, MACD, BBands)
# Using 'ta' library
# ============================================
# Limited indicators added manually later on increased using ta library
# ============================================
# Fix Close column to 1D Series
# ============================================
close_series = df["Close"].squeeze()

# RSI (14)
df["RSI"] = momentum.RSIIndicator(close_series, window=14).rsi()

# MACD
macd = trend.MACD(close=close_series, window_slow=26, window_fast=12, window_sign=9)
df["MACD"] = macd.macd()
df["MACD_signal"] = macd.macd_signal()
df["MACD_hist"] = macd.macd_diff()

# Bollinger Bands
bb = volatility.BollingerBands(close=close_series, window=20, window_dev=2)
df["BB_mid"] = bb.bollinger_mavg()
df["BB_upper"] = bb.bollinger_hband()
df["BB_lower"] = bb.bollinger_lband()
df["BB_width"] = df["BB_upper"] - df["BB_lower"]

df.dropna(inplace=True)
print("Tech Indicators Added:", df.shape)
df.tail()

# ============================================
# 4. Preprocessing
# Scaling + Train/Test Split
# ============================================

# Create target variable → next day's close
df["Target"] = df["Close"].shift(-1)
df.dropna(inplace=True)

# Feature set
features = df.drop("Target", axis=1)
target = df["Target"]

# Scale features with StandardScaler (or MinMaxScaler)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Train/test split (80/20)
split = int(len(df) * 0.8)
X_train_raw = scaled_features[:split]
X_test_raw = scaled_features[split:]
y_train = target[:split].values
y_test = target[split:].values

print("Train/Test Shapes:", X_train_raw.shape, X_test_raw.shape)

# ============================================
# 5. Create LSTM Sequences
# seq_len = number of past days used to predict next day's close
# ============================================

def create_sequences(X, y, seq_len=60):
    X_seq, y_seq = [], []
    for i in range(seq_len, len(X)):
        X_seq.append(X[i-seq_len:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

SEQ_LEN = 60

X_train, y_train = create_sequences(X_train_raw, y_train, SEQ_LEN)
X_test, y_test = create_sequences(X_test_raw, y_test, SEQ_LEN)

print("LSTM Dataset Shapes:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

# ============================================
# Final confirmation
# ============================================
print("Dataset ready for modeling.")


/tmp/ipython-input-766364992.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2012-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed

Data Loaded: (3204, 5)
Tech Indicators Added: (3171, 13)
Train/Test Shapes: (2536, 13) (634, 13)
LSTM Dataset Shapes:
X_train: (2476, 60, 13)
y_train: (2476,)
X_test: (574, 60, 13)
y_test: (574,)
Dataset ready for modeling.



/tmp/ipython-input-766364992.py:68: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  features = df.drop("Target", axis=1)


In [12]:
!pip install ta --quiet

In [30]:
import pandas as pd
import numpy as np

from ta.momentum import (
    RSIIndicator,
    StochasticOscillator,
    StochRSIIndicator,
    WilliamsRIndicator,
    ROCIndicator,
    AwesomeOscillatorIndicator,
    PercentagePriceOscillator,
    PercentageVolumeOscillator,
    TSIIndicator,
    UltimateOscillator
)

from ta.trend import (
    CCIIndicator,
    TRIXIndicator,
    KSTIndicator
)



class MomentumIndicators:
    """
    Momentum Indicators Class

    Purpose:
        Compute momentum-based indicators used for identifying strength,
        speed, and sustainability of price movements.

    Expected Input:
        DataFrame with columns: ['Open', 'High', 'Low', 'Close', 'Volume']

    Output:
        Individual methods return a Pandas Series.
        The `all()` method returns all indicators combined.
    """

    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()

        # --- FIX Yahoo Finance multi-index + (N,1) shaped columns ---
        if isinstance(self.df.columns, pd.MultiIndex):
            self.df.columns = self.df.columns.get_level_values(0)

        self.df = self.df.apply(lambda col: col.squeeze())

        self._validate_columns()

    def _validate_columns(self):
        """Ensures required OHLCV data is available."""
        required = ['Open', 'High', 'Low', 'Close', 'Volume']
        for col in required:
            if col not in self.df.columns:
                raise ValueError(f"Missing required column: {col}")

    # ======================================================================
    # 1. Relative Strength Index (RSI)
    # ======================================================================
    def rsi(self, window: int = 14) -> pd.Series:
        """
        Relative Strength Index (RSI)

        Description:
            Measures momentum by comparing average gains and losses.

        Interpretation:
            - RSI > 70 → Overbought (risk of pullback)
            - RSI < 30 → Oversold (risk of bounce)
            - RSI crossing 50 indicates trend direction
            - Divergences strongly indicate reversals

        Parameters:
            window: Default 14-period.

        Returns:
            RSI Series.
        """
        return RSIIndicator(close=self.df["Close"], window=window).rsi()

    # ======================================================================
    # 2. Stochastic Oscillator (%K, %D)
    # ======================================================================
    def stochastic_k(self, window: int = 14, smooth_window: int = 3) -> pd.Series:
        """
        Stochastic Oscillator %K

        Description:
            Compares current close to recent high-low range.

        Interpretation:
            - > 80 → Overbought
            - < 20 → Oversold
            - %K crossing above %D → Bullish signal
            - %K crossing below %D → Bearish signal

        Returns:
            %K Series.
        """
        return StochasticOscillator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"],
            window=window,
            smooth_window=smooth_window
        ).stoch()

    def stochastic_d(self, window: int = 14, smooth_window: int = 3) -> pd.Series:
        """
        Stochastic Oscillator %D (smoothed %K)

        Interpretation:
            - Used for confirming %K signals
            - More stable than %K

        Returns:
            %D Series.
        """
        return StochasticOscillator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"],
            window=window,
            smooth_window=smooth_window
        ).stoch_signal()

    # ======================================================================
    # 3. Stochastic RSI
    # ======================================================================
    def stoch_rsi(self, window: int = 14) -> pd.Series:
        """
        Stochastic RSI (StochRSI)

        Description:
            Stochastic oscillator applied to RSI instead of price.

        Interpretation:
            - > 0.8 → Overbought
            - < 0.2 → Oversold
            - More sensitive than RSI and Stochastic

        Returns:
            StochRSI Series.
        """
        return StochRSIIndicator(close=self.df["Close"], window=window).stochrsi()

    # ======================================================================
    # 4. Williams %R
    # ======================================================================
    def williams_r(self, window: int = 14) -> pd.Series:
        """
        Williams %R

        Description:
            Momentum oscillator similar to Stochastic.

        Interpretation:
            - > -20 → Overbought
            - < -80 → Oversold

        Returns:
            Williams %R Series.
        """
        return WilliamsRIndicator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"],
            lbp=window
        ).williams_r()

    # ======================================================================
    # 5. Rate of Change (ROC)
    # ======================================================================
    def roc(self, window: int = 12) -> pd.Series:
        """
        Rate of Change (ROC)

        Description:
            Measures percentage change over N periods.

        Interpretation:
            - Positive ROC → Strong momentum
            - Negative ROC → Bearish momentum
            - Extreme ROC spikes → Exhaustion / reversals

        Returns:
            ROC Series.
        """
        return ROCIndicator(close=self.df["Close"], window=window).roc()

    # ======================================================================
    # 6. Momentum Indicator
    # ======================================================================
    def momentum(self, window: int = 10) -> pd.Series:
        """
        Momentum Indicator

        Description:
            Measures the absolute price change over N periods.

        Interpretation:
            - Rising → Strengthening trend
            - Falling → Weakening trend

        Returns:
            Momentum Series.
        """
        return self.df["Close"].diff(window)

    # ======================================================================
    # 7. Ultimate Oscillator
    # ======================================================================
    def ultimate_oscillator(self) -> pd.Series:
        """
        Ultimate Oscillator

        Description:
            Combines 3 different timeframes (7, 14, 28)
            to avoid false signals from single-timeframe oscillators.

        Interpretation:
            - > 70 → Bullish / Overbought
            - < 30 → Bearish / Oversold

        Returns:
            UO Series.
        """
        return UltimateOscillator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"]
        ).ultimate_oscillator()

    # ======================================================================
    # 8. Commodity Channel Index (CCI)
    # ======================================================================
    def cci(self, window: int = 20) -> pd.Series:
        """
        Commodity Channel Index (CCI)

        Description:
            Measures momentum relative to a mean price.

        Interpretation:
            - > 100 → Strong bullish momentum
            - < -100 → Strong bearish momentum
            - Works well for spotting reversals and breakouts

        Returns:
            CCI Series.
        """
        return CCIIndicator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"],
            window=window
        ).cci()

    # ======================================================================
    # 9. TRIX
    # ======================================================================
    def trix(self, window: int = 15) -> pd.Series:
        """
        TRIX Indicator

        Description:
            Triple-smoothed EMA momentum oscillator.
            Filters noise extremely well.

        Interpretation:
            - TRIX > 0 → Uptrend momentum
            - TRIX < 0 → Downtrend momentum
            - Signal line crossovers are powerful

        Returns:
            TRIX Series.
        """
        return TRIXIndicator(close=self.df["Close"], window=window).trix()

    # ======================================================================
    # 10. Awesome Oscillator (AO)
    # ======================================================================
    def awesome(self) -> pd.Series:
        """
        Awesome Oscillator (AO)

        Description:
            Measures momentum using the difference between two SMAs
            of the mid-price (High+Low)/2.

        Interpretation:
            - AO > 0 → Bullish momentum
            - AO < 0 → Bearish momentum
            - Twin-peaks divergence used for reversals

        Returns:
            AO Series.
        """
        return AwesomeOscillatorIndicator(
            high=self.df["High"],
            low=self.df["Low"]
        ).awesome_oscillator()

    # ======================================================================
    # 11. PPO (Percentage Price Oscillator)
    # ======================================================================
    def ppo(self) -> pd.Series:
        """
        Percentage Price Oscillator (PPO)

        Description:
            Percentage-based MACD-like indicator.

        Interpretation:
            - Positive PPO → Uptrend momentum
            - Negative PPO → Downtrend momentum
            - More comparable across assets than MACD

        Returns:
            PPO Series.
        """
        return PercentagePriceOscillator(close=self.df["Close"]).ppo()

    # ======================================================================
    # 12. PVO (Percentage Volume Oscillator)
    # ======================================================================
    def pvo(self) -> pd.Series:
        """
        Percentage Volume Oscillator (PVO)

        Description:
            Measures momentum of volume.

        Interpretation:
            - Rising PVO → Increasing volume momentum (breakouts)
            - Falling PVO → Decreasing momentum (weakening trend)

        Returns:
            PVO Series.
        """
        return PercentageVolumeOscillator(volume=self.df["Volume"]).pvo()

    # ======================================================================
    # 13. Know Sure Thing (KST)
    # ======================================================================
    def kst(self) -> pd.Series:
        """
        Know Sure Thing (KST)

        Description:
            Multi-cycle momentum oscillator using
            4 different ROC calculations.

        Interpretation:
            - KST > Signal → Bullish
            - KST < Signal → Bearish
            - Good for identifying major trend momentum

        Returns:
            KST Series.
        """
        return KSTIndicator(close=self.df["Close"]).kst()

    # ======================================================================
    # 14. True Strength Index (TSI)
    # ======================================================================
    def tsi(self) -> pd.Series:
        """
        True Strength Index (TSI)

        Description:
            Measures momentum using double-smoothed price changes.

        Interpretation:
            - > 0 bullish
            - < 0 bearish
            - Crossovers and divergences are highly reliable

        Returns:
            TSI Series.
        """
        return TSIIndicator(close=self.df["Close"]).tsi()

    # ======================================================================
    # 15. KDJ Indicator (%J extension of stochastic)
    # ======================================================================
    def kdj(self) -> pd.Series:
        """
        KDJ Indicator

        Description:
            Extension of Stochastic Oscillator using:
                %J = 3*%K - 2*%D

        Interpretation:
            - %J > 100 → Overbought extreme
            - %J < 0 → Oversold extreme

        Returns:
            %J Series.
        """
        k = self.stochastic_k()
        d = self.stochastic_d()
        return 3 * k - 2 * d

    # ======================================================================
    # 16. Aggregated Output
    # ======================================================================
    def all(self) -> pd.DataFrame:
        """
        Returns all momentum indicators in a single DataFrame.

        Useful For:
            - ML feature engineering
            - Deep learning sequence modeling
            - Backtesting & factor research

        Returns:
            DataFrame containing every momentum indicator.
        """
        return pd.DataFrame({
            "RSI": self.rsi(),
            "Stoch_K": self.stochastic_k(),
            "Stoch_D": self.stochastic_d(),
            "Stoch_RSI": self.stoch_rsi(),
            "WilliamsR": self.williams_r(),
            "ROC": self.roc(),
            "Momentum": self.momentum(),
            "Ultimate_Osc": self.ultimate_oscillator(),
            "CCI": self.cci(),
            "TRIX": self.trix(),
            "Awesome": self.awesome(),
            "PPO": self.ppo(),
            "PVO": self.pvo(),
            "KST": self.kst(),
            "TSI": self.tsi(),
            "KDJ": self.kdj()
        })


import importlib

# List of all trend indicators we want to load
TREND_IMPORTS = [
    "SMAIndicator",
    "EMAIndicator",
    "WMAIndicator",
    "KAMAIndicator",
    "MACD",
    "MACD",
    "MACD",
    "ADXIndicator",
    "ADXIndicator",
    "ADXIndicator",
    "CCIIndicator",
    "IchimokuConvIndicator",
    "IchimokuBaseIndicator",
    "IchimokuAIndicator",
    "IchimokuBIndicator",
    "PSARIndicator",
    "TRIXIndicator",
    "KSTIndicator",
    "MassIndex",
    "VortexIndicator",
    "VortexIndicator"
]

# Module we import from
module_name = "ta.trend"

# Dictionary to store successfully loaded classes
loaded_trend_indicators = {}

print("\n🚀 Checking available imports from ta.trend...\n")

for indicator in TREND_IMPORTS:
    try:
        module = importlib.import_module(module_name)
        loaded_trend_indicators[indicator] = getattr(module, indicator)
        print(f"✔ AVAILABLE: {indicator}")
    except Exception as e:
        print(f"❌ NOT AVAILABLE: {indicator} — {type(e).__name__}: {e}")

# Export the working indicators into local namespace
globals().update(loaded_trend_indicators)

print("\n📌 Import test completed.")
print(f"📊 Total Available: {len(loaded_trend_indicators)} / {len(TREND_IMPORTS)}")

class TrendIndicators:
    """
    Trend Indicators Class

    Purpose:
        Computes trend-based indicators used for identifying long-term direction,
        trend reversals, breakouts, and trend strength.

    Requirements:
        DataFrame with columns:
            ['Open', 'High', 'Low', 'Close', 'Volume']
    """

    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()

        # --- FIX Yahoo multi-index & (N,1) shaped columns ---
        if isinstance(self.df.columns, pd.MultiIndex):
            self.df.columns = self.df.columns.get_level_values(0)
        self.df = self.df.apply(lambda col: col.squeeze())

    # ======================================================================
    # 1. Simple Moving Average (SMA)
    # ======================================================================
    def sma(self, window: int = 20) -> pd.Series:
        """
        SMA — Simple Moving Average

        Usage:
            Smooth trend direction.

        Interpretation:
            - Price > SMA → bullish regime
            - Price < SMA → bearish regime
            - SMA crossover (Golden Cross, Death Cross)

        ML value:
            Removes noise; essential baseline trend feature.
        """
        return SMAIndicator(close=self.df["Close"], window=window).sma_indicator()

    # ======================================================================
    # 2. Exponential Moving Average (EMA)
    # ======================================================================
    def ema(self, window: int = 20) -> pd.Series:
        """
        EMA — Exponential Moving Average

        Faster moving average.

        Interpretation:
            - Reacts faster than SMA.
            - Popular for crossover strategies.
        """
        return EMAIndicator(close=self.df["Close"], window=window).ema_indicator()

    # ======================================================================
    # 3. WMA — Weighted Moving Average
    # ======================================================================
    def wma(self, window: int = 20) -> pd.Series:
        """
        Weighted Moving Average (WMA)

        Larger weights on recent prices.
        """
        return WMAIndicator(close=self.df["Close"], window=window).wma()

    # ======================================================================
    # 4. HMA — Hull Moving Average (manual)
    # ======================================================================
    def hma(self, window: int = 20) -> pd.Series:
        """
        HMA — Hull Moving Average

        Equation:
            HMA = WMA(2*WMA(n/2) - WMA(n), sqrt(n))

        Purpose:
            Fast + smooth moving average.

        Interpretation:
            Excellent for momentum detection with low lag.
        """
        half = window // 2
        sqrt = int(np.sqrt(window))

        wma_half = self.df["Close"].rolling(half).mean()
        wma_full = self.df["Close"].rolling(window).mean()
        hull_input = 2 * wma_half - wma_full
        return hull_input.rolling(sqrt).mean()

    # ======================================================================
    # 5. KAMA — Kaufman's Adaptive Moving Average
    # ======================================================================
    def kama(self, window: int = 10) -> pd.Series:
        """
        KAMA — Kaufman Adaptive Moving Average

        Adapts to volatility:
            - Trendy markets → fast EMA-like
            - Choppy markets → slow SMA-like
        """
        return KAMAIndicator(close=self.df["Close"], window=window).kama()

    # ======================================================================
    # 6. VWMA — Volume Weighted Moving Average (manual)
    # ======================================================================
    def vwma(self, window: int = 20) -> pd.Series:
        """
        VWMA — Volume Weighted Moving Average

        Weights each price by volume.
        Stronger than SMA in high-volume markets.
        """
        pv = self.df["Close"] * self.df["Volume"]
        return pv.rolling(window).sum() / self.df["Volume"].rolling(window).sum()

    # ======================================================================
    # 7. MACD (Line, Signal, Histogram)
    # ======================================================================
    def macd_line(self) -> pd.Series:
        """
        MACD Line = EMA(12) - EMA(26)
        """
        return MACD(close=self.df["Close"]).macd()

    def macd_signal(self) -> pd.Series:
        """MACD Signal Line"""
        return MACD(close=self.df["Close"]).macd_signal()

    def macd_hist(self) -> pd.Series:
        """MACD Histogram"""
        return MACD(close=self.df["Close"]).macd_diff()

    # ======================================================================
    # 8. ADX — Average Directional Index
    # ======================================================================
    def adx(self, window: int = 14) -> pd.Series:
        """
        ADX — Trend Strength Indicator

        Interpretation:
            ADX < 20 → Weak or no trend
            ADX > 25 → Strong trend
            ADX > 35 → Very strong trend
        """
        return ADXIndicator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"],
            window=window
        ).adx()

    # ======================================================================
    # 9. DMI — Directional Movement Indicators
    # ======================================================================
    def plus_di(self) -> pd.Series:
        """+DI — Bullish directional movement"""
        return ADXIndicator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"]
        ).adx_pos()

    def minus_di(self) -> pd.Series:
        """–DI — Bearish directional movement"""
        return ADXIndicator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"]
        ).adx_neg()

    # ======================================================================
    # 10. Aroon Up / Down
    # ======================================================================
    def aroon_up(self, window: int = 25) -> pd.Series:
        """
        Aroon Up

        Measures time since last highest high.
        """
        highest = self.df["High"].rolling(window).apply(lambda x: window - np.argmax(x), raw=True)
        return 100 * (highest / window)

    def aroon_down(self, window: int = 25) -> pd.Series:
        """
        Aroon Down

        Measures time since last lowest low.
        """
        lowest = self.df["Low"].rolling(window).apply(lambda x: window - np.argmin(x), raw=True)
        return 100 * (lowest / window)

    # ======================================================================
    # 11. CCI — Commodity Channel Index
    # ======================================================================
    def cci(self, window: int = 20) -> pd.Series:
        """Trend identification & overbought/oversold signals."""
        return CCIIndicator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"],
            window=window
        ).cci()

    # ======================================================================
    # 12. Ichimoku Cloud (Full Components)
    # ======================================================================
    def ichimoku_conversion(self) -> pd.Series:
        return IchimokuConvIndicator(
            high=self.df["High"], low=self.df["Low"]
        ).ichimoku_conversion_line()

    def ichimoku_base(self) -> pd.Series:
        return IchimokuBaseIndicator(
            high=self.df["High"], low=self.df["Low"]
        ).ichimoku_base_line()

    def ichimoku_a(self) -> pd.Series:
        return IchimokuAIndicator(
            high=self.df["High"], low=self.df["Low"]
        ).ichimoku_a()

    def ichimoku_b(self) -> pd.Series:
        return IchimokuBIndicator(
            high=self.df["High"], low=self.df["Low"]
        ).ichimoku_b()

    # ======================================================================
    # 13. Parabolic SAR
    # ======================================================================
    def parabolic_sar(self) -> pd.Series:
        """SAR — stop & reverse trend indicator"""
        return PSARIndicator(
            high=self.df["High"],
            low=self.df["Low"],
            close=self.df["Close"]
        ).psar()

    # ======================================================================
    # 14. TRIX
    # ======================================================================
    def trix(self) -> pd.Series:
        """TRIX — Triple EMA Momentum Trend Index"""
        return TRIXIndicator(close=self.df["Close"]).trix()

    # ======================================================================
    # 15. KST — Know Sure Thing
    # ======================================================================
    def kst(self) -> pd.Series:
        """KST — Multi-timeframe trend oscillator"""
        return KSTIndicator(close=self.df["Close"]).kst()

    # ======================================================================
    # 16. Mass Index (Reversal predictor)
    # ======================================================================
    def mass_index(self) -> pd.Series:
        return MassIndex(high=self.df["High"], low=self.df["Low"]).mass_index()

    # ======================================================================
    # 17. Vortex (+VI, –VI)
    # ======================================================================
    def vortex_plus(self) -> pd.Series:
        return VortexIndicator(
            high=self.df["High"], low=self.df["Low"], close=self.df["Close"]
        ).vortex_indicator_pos()

    def vortex_minus(self) -> pd.Series:
        return VortexIndicator(
            high=self.df["High"], low=self.df["Low"], close=self.df["Close"]
        ).vortex_indicator_neg()

    # ======================================================================
    # 18. DPO — Detrended Price Oscillator (manual)
    # ======================================================================
    def dpo(self, window: int = 20) -> pd.Series:
        """
        DPO — Detrended Price Oscillator

        Removes long-term cycles to expose short-term cycles.
        """
        shift = int(window / 2 + 1)
        sma = self.df["Close"].rolling(window).mean()
        return self.df["Close"].shift(shift) - sma

    # ======================================================================
    # 19. TEMA — Triple EMA (manual)
    # ======================================================================
    def tema(self, window: int = 20) -> pd.Series:
        """
        TEMA — Triple Exponential Moving Average
        """
        ema1 = self.df["Close"].ewm(span=window).mean()
        ema2 = ema1.ewm(span=window).mean()
        ema3 = ema2.ewm(span=window).mean()
        return 3 * (ema1 - ema2) + ema3

    # ======================================================================
    # 20. Aggregate all indicators
    # ======================================================================
    def all(self) -> pd.DataFrame:
        """
        Returns every trend indicator as a DataFrame.
        Used for ML feature engineering.

        Includes:
            - Moving averages
            - MACD components
            - Trend strength (ADX, DMI)
            - Cloud signals
            - Vortex, Mass Index, TRIX, KST
        """
        return pd.DataFrame({
            "SMA20": self.sma(),
            "EMA20": self.ema(),
            "WMA20": self.wma(),
            "HMA20": self.hma(),
            #"KAMA10": self.kama(),
            "VWMA20": self.vwma(),
            "MACD": self.macd_line(),
            "MACD_Signal": self.macd_signal(),
            "MACD_Hist": self.macd_hist(),
            "ADX": self.adx(),
            "+DI": self.plus_di(),
            "-DI": self.minus_di(),
            "AroonUp": self.aroon_up(),
            "AroonDown": self.aroon_down(),
            "CCI": self.cci(),
            #"Ich_Conversion": self.ichimoku_conversion(),
            #"Ich_Base": self.ichimoku_base(),
            #"Ich_A": self.ichimoku_a(),
            #"Ich_B": self.ichimoku_b(),
            "Parabolic_SAR": self.parabolic_sar(),
            "TRIX": self.trix(),
            "KST": self.kst(),
            "Mass_Index": self.mass_index(),
            "Vortex_Pos": self.vortex_plus(),
            "Vortex_Neg": self.vortex_minus(),
            "DPO": self.dpo(),
            "TEMA": self.tema(),
        })



🚀 Checking available imports from ta.trend...

✔ AVAILABLE: SMAIndicator
✔ AVAILABLE: EMAIndicator
✔ AVAILABLE: WMAIndicator
❌ NOT AVAILABLE: KAMAIndicator — AttributeError: module 'ta.trend' has no attribute 'KAMAIndicator'
✔ AVAILABLE: MACD
✔ AVAILABLE: MACD
✔ AVAILABLE: MACD
✔ AVAILABLE: ADXIndicator
✔ AVAILABLE: ADXIndicator
✔ AVAILABLE: ADXIndicator
✔ AVAILABLE: CCIIndicator
❌ NOT AVAILABLE: IchimokuConvIndicator — AttributeError: module 'ta.trend' has no attribute 'IchimokuConvIndicator'
❌ NOT AVAILABLE: IchimokuBaseIndicator — AttributeError: module 'ta.trend' has no attribute 'IchimokuBaseIndicator'
❌ NOT AVAILABLE: IchimokuAIndicator — AttributeError: module 'ta.trend' has no attribute 'IchimokuAIndicator'
❌ NOT AVAILABLE: IchimokuBIndicator — AttributeError: module 'ta.trend' has no attribute 'IchimokuBIndicator'
✔ AVAILABLE: PSARIndicator
✔ AVAILABLE: TRIXIndicator
✔ AVAILABLE: KSTIndicator
✔ AVAILABLE: MassIndex
✔ AVAILABLE: VortexIndicator
✔ AVAILABLE: VortexIndicator

📌 

In [31]:
trend = TrendIndicators(df)
momentum = MomentumIndicators(df)

# Get indicator groups
trend_df = trend.all()
momentum_df = momentum.all()


# Base OHLCV
base_df = df.copy()
base_df = base_df[['Open', 'High', 'Low', 'Close', 'Volume']]
base_df = base_df.apply(lambda c: c.squeeze())

print(base_df.columns)

base_df.head()

/usr/local/lib/python3.12/dist-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


MultiIndex([(  'Open', 'RELIANCE.NS'),
            (  'High', 'RELIANCE.NS'),
            (   'Low', 'RELIANCE.NS'),
            ( 'Close', 'RELIANCE.NS'),
            ('Volume', 'RELIANCE.NS')],
           names=['Price', 'Ticker'])


Price,Open,High,Low,Close,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2012-02-17,169.869534,170.768425,167.420834,169.073959,35922700
2012-02-21,168.929326,175.386878,168.433387,174.312332,36457910
2012-02-22,175.231896,177.174325,171.533024,172.214935,25146771
2012-02-23,171.925624,175.624510,170.809765,173.496109,19105924
2012-02-24,172.235602,173.930068,168.309417,169.435608,18746068


In [32]:
# Combine everything
full_df = pd.concat(
    [base_df, trend_df, momentum_df],
    axis=1
)

print(full_df.columns)

full_df.head()



Index([  ('Open', 'RELIANCE.NS'),   ('High', 'RELIANCE.NS'),
          ('Low', 'RELIANCE.NS'),  ('Close', 'RELIANCE.NS'),
       ('Volume', 'RELIANCE.NS'),                   'SMA20',
                         'EMA20',                   'WMA20',
                         'HMA20',                  'VWMA20',
                          'MACD',             'MACD_Signal',
                     'MACD_Hist',                     'ADX',
                           '+DI',                     '-DI',
                       'AroonUp',               'AroonDown',
                           'CCI',           'Parabolic_SAR',
                          'TRIX',                     'KST',
                    'Mass_Index',              'Vortex_Pos',
                    'Vortex_Neg',                     'DPO',
                          'TEMA',                     'RSI',
                       'Stoch_K',                 'Stoch_D',
                     'Stoch_RSI',               'WilliamsR',
                        

,"(Open, RELIANCE.NS)","(High, RELIANCE.NS)","(Low, RELIANCE.NS)","(Close, RELIANCE.NS)","(Volume, RELIANCE.NS)",SMA20,EMA20,WMA20,HMA20,VWMA20,...,Momentum,Ultimate_Osc,CCI,TRIX,Awesome,PPO,PVO,KST,TSI,KDJ
Date,,,,,,,,,,,,,,,,,,,,,
2012-02-17,169.869534,170.768425,167.420834,169.073959,35922700,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-21,168.929326,175.386878,168.433387,174.312332,36457910,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-22,175.231896,177.174325,171.533024,172.214935,25146771,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-23,171.925624,175.624510,170.809765,173.496109,19105924,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-24,172.235602,173.930068,168.309417,169.435608,18746068,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
print(f"Length of full_df: {len(full_df)}")
# Remove all missing values
full_df.replace([np.inf, -np.inf], np.nan, inplace=True)
#full_df.dropna(inplace=True)

print("Final Feature Set Shape:", full_df.shape)
full_df.head()




Length of full_df: 3170
Final Feature Set Shape: (3170, 43)


,"(Open, RELIANCE.NS)","(High, RELIANCE.NS)","(Low, RELIANCE.NS)","(Close, RELIANCE.NS)","(Volume, RELIANCE.NS)",SMA20,EMA20,WMA20,HMA20,VWMA20,...,Momentum,Ultimate_Osc,CCI,TRIX,Awesome,PPO,PVO,KST,TSI,KDJ
Date,,,,,,,,,,,,,,,,,,,,,
2012-02-17,169.869534,170.768425,167.420834,169.073959,35922700,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-21,168.929326,175.386878,168.433387,174.312332,36457910,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-22,175.231896,177.174325,171.533024,172.214935,25146771,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-23,171.925624,175.624510,170.809765,173.496109,19105924,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-24,172.235602,173.930068,168.309417,169.435608,18746068,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
full_df.tail()

,"(Open, RELIANCE.NS)","(High, RELIANCE.NS)","(Low, RELIANCE.NS)","(Close, RELIANCE.NS)","(Volume, RELIANCE.NS)",SMA20,EMA20,WMA20,HMA20,VWMA20,...,Momentum,Ultimate_Osc,CCI,TRIX,Awesome,PPO,PVO,KST,TSI,KDJ
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-23,1210.166735,1222.318155,1208.373847,1217.437744,10052824,1272.206433,1260.626930,1258.317509,1244.777869,1270.761973,...,-72.560181,39.236269,-152.177223,-0.154682,-42.010455,-1.689169,-0.743724,-27.933230,-20.775090,34.854150
2024-12-24,1217.437703,1228.642950,1216.142826,1217.885864,6734917,1268.573444,1256.556352,1253.144121,1230.573351,1268.929167,...,-61.852905,31.174994,-119.718534,-0.169342,-44.706597,-1.755202,-5.355793,-32.510440,-21.668815,25.833550
2024-12-26,1219.379892,1222.816119,1209.419672,1211.710571,10016178,1264.756195,1252.285325,1247.728610,1218.907451,1266.361975,...,-61.404663,30.587520,-117.790127,-0.183639,-48.296230,-1.827636,-7.291019,-36.788170,-22.758331,6.865765
2024-12-27,1213.453688,1223.015476,1212.158811,1216.192749,7000397,1262.278595,1248.847937,1243.103520,1210.319881,1265.168661,...,-41.683472,33.152372,-98.644952,-0.196077,-48.495419,-1.835742,-10.693295,-40.835558,-23.212527,20.964968
2024-12-30,1211.561187,1218.334064,1203.294156,1205.883789,8818766,1258.219806,1244.756113,1237.732586,1202.474344,1262.291402,...,-61.902832,29.618084,-102.505053,-0.207952,-46.600933,-1.888538,-12.300481,-45.563503,-24.211394,3.303134


🚀 Next Steps:

➤ Create Target Column (returns, classification, or next-day close)

➤ Scale the data (StandardScaler / MinMaxScaler)

➤ Convert to LSTM sequences

➤ Train ML or DL models

➤ Save features to CSV / feather

➤ Feature selection / PCA